In [19]:
#import libraries
import cv2
import numpy as np
import glob
import random
from PIL import Image  
import PIL  

In [17]:
#store the weights,cfg,test image folder in a single dir 
#change the directory q
import os
#os.chdir(r'path where weights images cfg files are saved')


In [16]:

net = cv2.dnn.readNet("yolov3_traffic_train_2000.weights", "yolov3_ts_test.cfg")


# Name custom object
classes = ["prohibitory","danger","mandatory","other"]

# Images path
images_path = glob.glob(r"test_images\*.jpg")

count=0
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    count=count+1
    
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            #print(x,y,w,h)
            print(img_path.split('\\')[-1],x,y,w,h)
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y-3), font, 3, color, 2)
    filename='output'+str(count)+'.jpg'      
    cv2.imwrite(filename, img) 
    
    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()

0
0
00222.jpg 392 140 15 15
